In [17]:
# Aaron Frost 2025

# let's learn about how to profile CUDA kernels - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#  I'm following along with https://www.youtube.com/watch?v=LuhJEEJQgUM&ab_channel=GPUMODE

# in order to profile individual CUDA operations (kernels), we can't use the python time module.
# this is because CUDA is ASYNC!
# if you want to profile an operation you might use:
#   start = torch.cuda.Event(enable_timing=True)  -- creates a start event
#   end = torch.cuda.Event(enable_timing=True)    -- creates an end event

# you also have to warm up CUDA before profiling
#   the first time you call CUDA in a pytorch function it will initialize, so we want to get that out of the way first before starting a timer.

# start.record() -- start the timer
# // execute the function
# end.record()   -- post the end event
#
# torch.cuda.synchronize()  -- AKA await the completion of the kernel

# the time it took:
# time = start.elapsed_time(end)

In [18]:
# what is torch autograd profiler?  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# built in pytorch profiler tells you how much time each kernel took on CPU and GPU
# gives you callstack, with the time it took at each method in the stack.

# with torch.autograd.profiler.profile(use_cuda=True) as profiler:
  # // do stuff that needs profiling

# print out the table to see what the most time consuming kernels are
# print(profiler.key_averages().table(sort_by="cuda_time_total", row_limit=10))

In [19]:
# what is pytorch profiler?  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#
# visual profiler
# doesn't give you debugging for kernel internals
# JSON file you can drag and drop into google chrome.
# you can see the CUDA kernels on teh pytorch github repo by looking for .cu

In [20]:
# how to integrate custom CUDA kernel in PyTorch

# basically load a C++ function in a python program
#    easiest way is
#       from torch.utils.cpp_extension import load_inline


# Ninja is required for this, so we need to execute this command in the runtime terminal:
# apt-get install ninja-build

# then you can write the .cu source inline:
# ex.
import torch
from torch.utils.cpp_extension import load_inline

cpp_source = """
std::string hello_world() {
  return "Hello World!";
}
"""

# under the hood this codegens a makefile to run a compiler and produce CPP output files and binds them to python using PYBIND
my_module = load_inline(
    name='my_module',
    cpp_sources=[cpp_source],
    functions=['hello_world'],
    verbose=True)

print(my_module.hello_world())

# https://www.youtube.com/watch?v=-6_CvTdzMRY&ab_channel=MarkSaroufim

Hello World!


Using /root/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
No modifications detected for re-loaded extension module my_module, skipping build step...
Loading extension module my_module...


In [21]:
# a lot of machine learning progress comes down to a "bag of tricks" people in the ML community know about to make models converge faster
# one trick is mixed-precision
# all you have to do is make the weights and inputs half precision...

# floating point data type has bits for mantissa and exponent (binary scientific notation)
# model size is a proxy for training time required.

# one more subtle thing you have to do:
#
# example: batch normalization - make sure outputs (activations) at any individual layer are not too big or too small:
#   this is crucial to ensure different features with varying ranges have a similar scale.
#   when you do this, using a lower precision would cause a problem:
#     smaller floating point types (less bits) means it can't have as many decimal points... so we're losing information with each batch norm
# introducing mixed precision: (loss-gradient scaling)
#   maintain copy of single precision (32 bit) floats
#       copy to half precision (16 bit) -> do foward propagation, multiply by scaling factor (to make it larger / get rid of decimal points),
#            do backprop, multiply weight gradient by 1/scaling factor, to make it smaller again

In [22]:
# USING CUDA:
#
# you can move a tensor to the GPU using .cuda()
# GPU only works well for tasks that can be broken down into many smaller parallel tasks, like traning neural networks

# common CUDA optimizations:
#
# Memory coalescing:
#   https://youtu.be/mLxZyWOI340?si=A4Kbj-OZvrLY8Jf-
#   GPU is most effienct when threads read or write contiguous global memory locations
#       - coalesced access - as opposed to strided (stride between each access) - reduces the number of memory transactions required
#   To programmers, a tensor might look like a square, but in RAM, it's a single linear set of addresses.
#     It's a perf optimization to SHARE memory access.
#     When each thread needs to access a different col of a matrix, it is more optimal than if each row needs to be accessed.
#         --> for this reason you might transpose the rows and cols.


# when you run a kernel, you define the block and how many threads are in the block, and grid layout (how many blocks)
# each thread block is assigned to a streaming multiprocessor - each can process a number of threads


# Shared memory:
#     Shared memory is a fast, user-managed memory that is shared among all threads in the same thread block.
#     Declare shared memory __shared__ float sharedArray[BLOCK_SIZE];


#     Example.. Matrix multiplication
#       Things to look out for: Bank conflicts degrading perf, limited memory size, ensuring to sync the threads.
"""
__global__ void matrixMul(float *A, float *B, float *C, int N) {
    __shared__ float Asub[TILE_SIZE][TILE_SIZE];
    __shared__ float Bsub[TILE_SIZE][TILE_SIZE];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = blockIdx.y * TILE_SIZE + ty;
    int col = blockIdx.x * TILE_SIZE + tx;

    float value = 0;

    for (int i = 0; i < N / TILE_SIZE; ++i) {
        // Load tiles into shared memory
        Asub[ty][tx] = A[row * N + (i * TILE_SIZE + tx)];
        Bsub[ty][tx] = B[(i * TILE_SIZE + ty) * N + col];
        __syncthreads(); // sync to ensure data-consistency

        // Perform multiplication
        for (int j = 0; j < TILE_SIZE; ++j) {
            value += Asub[ty][j] * Bsub[j][tx];
        }
        __syncthreads();
    }

    // Write result to global memory
    C[row * N + col] = value;
}"""

'\n__global__ void matrixMul(float *A, float *B, float *C, int N) {\n    __shared__ float Asub[TILE_SIZE][TILE_SIZE];\n    __shared__ float Bsub[TILE_SIZE][TILE_SIZE];\n\n    int tx = threadIdx.x;\n    int ty = threadIdx.y;\n    int row = blockIdx.y * TILE_SIZE + ty;\n    int col = blockIdx.x * TILE_SIZE + tx;\n\n    float value = 0;\n\n    for (int i = 0; i < N / TILE_SIZE; ++i) {\n        // Load tiles into shared memory\n        Asub[ty][tx] = A[row * N + (i * TILE_SIZE + tx)];\n        Bsub[ty][tx] = B[(i * TILE_SIZE + ty) * N + col];\n        __syncthreads(); // sync to ensure data-consistency\n\n        // Perform multiplication\n        for (int j = 0; j < TILE_SIZE; ++j) {\n            value += Asub[ty][j] * Bsub[j][tx];\n        }\n        __syncthreads();\n    }\n\n    // Write result to global memory\n    C[row * N + col] = value;\n}'

In [23]:
# Anyways, let's now pull in our flower dataset, and write a custom load_inline cuda extension for batch normalization method to use
# fp8 mixed precision.

In [24]:
# currently working on this part...

import torch
from torch.utils.cpp_extension import load_inline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Normalize and split the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Define a simple MLP
class IrisMLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(IrisMLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.bn = torch.nn.BatchNorm1d(hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

# Load Inline CUDA Extension
batch_norm_extension = load_inline(
    name="batch_norm_fp8",
    cpp_sources="""
#include <torch/extension.h>
#include "cuda_runtime.h"
#include <ATen/cuda/CUDAContext.h>

// global function runs on the GPU, and is called from the CPU
__global__ void batch_norm_fp8_kernel(const float* __restrict__ input,
                                      float* __restrict__ output,
                                      const int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = input[idx] * 0.125;  // Simulating FP8
    }
}

torch::Tensor batch_norm_fp8(torch::Tensor input) {
    auto output = torch::empty_like(input);
    const int threads = 1024;
    const int blocks = (input.numel() + threads - 1) / threads;

    batch_norm_fp8_kernel<<<blocks, threads>>>(
        input.data_ptr<float>(), output.data_ptr<float>(), input.numel());
    return output;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("batch_norm_fp8", &batch_norm_fp8, "BatchNorm FP8 kernel");
}
""",
    functions=["batch_norm_fp8"],
    extra_cuda_cflags=["--use_fast_math"],
)

# Define a custom batch normalization function using the CUDA extension
class CustomBatchNormFP8(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return batch_norm_extension.batch_norm_fp8(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clone()

# Replace the batch norm in the MLP model
class CustomIrisMLP(IrisMLP):
    def __init__(self, input_size, hidden_size, num_classes):
        super(CustomIrisMLP, self).__init__(input_size, hidden_size, num_classes)
        self.bn = CustomBatchNormFP8.apply

# Train and profile the model
model = CustomIrisMLP(input_size=4, hidden_size=8, num_classes=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

# Profiling with autograd profiler
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

RuntimeError: Error building extension 'batch_norm_fp8_v2': [1/2] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=batch_norm_fp8_v2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.11/dist-packages/torch/include -isystem /usr/local/lib/python3.11/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.11/dist-packages/torch/include/TH -isystem /usr/local/lib/python3.11/dist-packages/torch/include/THC -isystem /usr/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /root/.cache/torch_extensions/py311_cu121/batch_norm_fp8/main.cpp -o main.o 
FAILED: main.o 
c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=batch_norm_fp8_v2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.11/dist-packages/torch/include -isystem /usr/loca